# Initial Data cleaning


In [ ]:
import urllib.request
import re
import pandas as pd
import numpy as np
import csv


In [ ]:
# mounting a drive folder to save the csv files :)
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
def convertToList(url):
  file = urllib.request.urlopen(url)
  return list(map(lambda line : line.decode("utf-8")[:-2], file.readlines()))

# convertToList("http://dcdd89a779ea.ngrok.io/conv(18).txt")[:20]

In [ ]:
def removeBotName(rlist):
  pattern = re.compile("(?!BracU.*Bot$)")
  return list(filter(pattern.match, rlist))

# removeBotName(convertToList("http://dcdd89a779ea.ngrok.io/conv(18).txt"))[:20]

In [ ]:
def removeDateTime(rlist):
  pattern = re.compile("(?!.*\d\d?:\d\d\s?(AM|PM)?)")
  return list(filter(pattern.match, rlist))

# removeDateTime(removeBotName(convertToList("http://dcdd89a779ea.ngrok.io/conv(18).txt")))

In [ ]:
def dropHeader(rlist):
  # drop heading irrelevant items
   pattern = re.compile("(?!^\t.*|Seen by.*|Get Started|^help$)", re.IGNORECASE)
   pattern2 = re.compile("(?!You\scan\swrite.*anytime.*$|[1-5]\.\s.*\?$|As per your request.*)")
   
   return list(filter(pattern2.match, list(filter(pattern.match, rlist))))

# dropHeader(removeDateTime(removeBotName(convertToList("http://dcdd89a779ea.ngrok.io/conv(18).txt"))))[:10]

In [ ]:
# Put the name in global varible for latter use
_name = ""
def name_match(n):
  return n != _name

def dropName(rlist):
  
  # get name from the first index
  global _name 
  _name = rlist.pop(0)
  #print("----------->"+name)
  # filter the name from the list
  return list(filter(name_match, rlist))

# dropName(dropHeader(removeDateTime(removeBotName(convertToList("http://dcdd89a779ea.ngrok.io/conv(18).txt")))))[:10]

# Dataset creation

In [ ]:
def convertToCsv(rlist):
  # join pair of lines using comma
  question = []
  answer = []

  for i in range(len(rlist)):
    if i%2==0:
        question.append(rlist[i])
    else:
        answer.append(rlist[i])
  
  dick = {
      'question' : question, 
      'answer' : answer
  }

  df = pd.DataFrame(dick)
  df.to_csv('/content/drive/My Drive/Colab Notebooks/422_project_csv_files/csv_data2.csv',index=False)


# convertToCsv(dropName(dropHeader(removeDateTime(removeBotName(convertToList(url))))))

NameError: ignored

In [ ]:
def cleaner(url):
  # need to modify convert to list to take url parameter
  return dropName(dropHeader(removeDateTime(removeBotName(convertToList(url)))))

# final_list = []
# total_dropped = 0
# for i in range(2,80):
#     url ="http://dcdd89a779ea.ngrok.io/conv({}).txt".format(i)
#     c_list = cleaner(url)
#     if len(c_list) % 2 != 0:
#       print("dropped #" + str(i))
#       total_dropped = total_dropped + 1
#       continue
#     final_list.extend(c_list)
  
# final_list[:20]

In [ ]:
# final_list.pop(2602)
# final_list[2600:2610]

In [ ]:
print(len(final_list))
print(str(final_list[2600:2610]))
convertToCsv(final_list)
#cleaned_data = convertToCsv(dropName(dropHeader(removeDateTime(removeBotName(convertToList())))))
#cleaned_data

NameError: ignored

In [ ]:
def splitResponse():
  # split failcases and successes
  # in returns two lists fail_case and success_case and creates two csv files in the drive of those lists  
  fail_case = []
  success_case = []
  with open("/content/drive/My Drive/Colab Notebooks/422_project_csv_files/csv_data2.csv",'r') as csv_file:
      reader = csv.reader(csv_file)
      with open("/content/drive/My Drive/Colab Notebooks/422_project_csv_files/success.csv",'w',newline='') as success:
          suc_write = csv.writer(success)
          
      
          with open("/content/drive/My Drive/Colab Notebooks/422_project_csv_files/failcase.csv",'w',newline='') as failcase:
              fail_write = csv.writer(failcase)
              fail_write.writerow(('question','answer'))
              fail_case.append(('question','answer'))
              suc_write.writerow(('question','answer'))
              success_case.append(('question','answer'))
              
              next(reader)
              for line in reader:
                  pattern = re.compile("Sorry can't understand yo.*|There is no information found regarding your qu.*")
                  if pattern.match(line[1]):
                      fail_write.writerow(line)
                      fail_case.append( tuple(line) )
                  else:
                      suc_write.writerow(line)
                      success_case.append(tuple(line))
    
  
  return (fail_case, success_case)

splitResponse()

([('question', 'answer'),
  ('San', "Sorry can't understand you."),
  ('mns faculty', "Sorry can't understand you."),
  ('Room numner of RAA', "Sorry can't understand you."),
  ('Email of ahr', "Sorry can't understand you."),
  ('Room number of IBU',
   'There is no information found regarding your query.'),
  ('Room number of EHK',
   'There is no information found regarding your query.'),
  ('Ehsanul Kabir', "Sorry can't understand you."),
  ('Aar', "Sorry can't understand you."),
  ('AAR', "Sorry can't understand you."),
  ('Annajiat', "Sorry can't understand you."),
  ('Room no of ehsanul kabir', "Sorry can't understand you."),
  ('Medical centre', "Sorry can't understand you."),
  ('Email id of SKF', 'There is no information found regarding your query.'),
  ('Email id of SAD', 'There is no information found regarding your query.'),
  ('Email id of WAA', 'There is no information found regarding your quer'),
  ('Sej', "Sorry can't understand you."),
  ('Mlr email id', "Sorry can't u

# Labeling
We identified the types of questions by `parameter` and `intent`. We will label the types of questions.

In [ ]:
_types = ["mail_id", "room_number", "teacher_takes", "full_name", "course_taken", "greetings", "initial"]
_patterns = {
    "mail_id": "Here\sis\sthe\semail\saddress.*",
    "room_number": ".*s(i|ea)ts in UB.*",
    "full_name": "Full\sname\s.*",
    "teacher_takes": "\w\w\w\stakes.*",
    "course_taken": "\w\w\w\d\d\d\sis\staken.*",
    "initial": "Initial of .*"
} 

def label():
  vari = 10
  successes = splitResponse()[1][1:]
  labels = []
  for pattern in _patterns:
    re_pat = re.compile(_patterns[pattern])
    for entry in successes:
      if re_pat.match(entry[1]):
        labels.append((entry[0], pattern))
      # else:
        # print(entry)
        
  return labels

questions = list(map(lambda x: x[0],label())) # labelled set (ques, label) -> [ques1, ques2]
successes = splitResponse()[1][1:] # 
for entry in successes: #(ques, answer)
  if entry[0] not in questions:
    print(entry[1])
print(str(len(label())) + " is processed out of " + str(len(splitResponse()[1][1:])))

_labelled = label()
label()

Welcome. 😊
What's your query? 😀
Welcome. 😊
What's your query? 😀
What's your query? 😀
What's your query? 😀
Welcome.
What's your query? 😀
What's your query? 😀
What's your query? 😀
Sent by Robin Mollah
Welcome. 😊
Sorry can't understand y
What's your query? 😀
What's your query? 😀
Welcome. 😊
Welcome. 😊
Welcome. 😊
357 is processed out of 375


[('Email id of AHR', 'mail_id'),
 ('Email id of msi', 'mail_id'),
 ('Email id of nazmus sakeef', 'mail_id'),
 ('Email id of ach', 'mail_id'),
 ('Email id of arf', 'mail_id'),
 ('Email id of szi?', 'mail_id'),
 ('email id of sro?', 'mail_id'),
 ('Mail id of salman sir?', 'mail_id'),
 ('Mail id of annajiat', 'mail_id'),
 ('Mail id of ishita miss?', 'mail_id'),
 ('Email id of mht?', 'mail_id'),
 ('email id of sro?', 'mail_id'),
 ('email id of szn?', 'mail_id'),
 ('mail id of mih', 'mail_id'),
 ('email id of szn?', 'mail_id'),
 ('email id of ish', 'mail_id'),
 ('Email id of zahid hussain mohammed al deen sir?', 'mail_id'),
 ('email id of mrn?', 'mail_id'),
 ('Email ID of SRO', 'mail_id'),
 ('Email ID of mmm', 'mail_id'),
 ('May I have the email ID of AHR?', 'mail_id'),
 ('Email ID of ARK!', 'mail_id'),
 ('May I have the email address of DZK', 'mail_id'),
 ('May I have the email address of RYB', 'mail_id'),
 ('Email id of mrn', 'mail_id'),
 ('Email id of arf', 'mail_id'),
 ('Email id of srj

# Preparation

In [ ]:
# all initial, course codes and other parameters will be changed to default_value for SVM's classification
# "mail id of aar" => [20, 10, 0, -1]
# list of words = vocabulary
# vocab =["mail", "room", "id", "of", "dzk", "aar"]
# "mail id of aar" => [1, 0, 1, 1, 0, 1]

def cleanParams():
  entries = label()
  cleaned = []
  for entry in entries:
    # remove initials
    if entry[0] == "Hi":
       continue
    pattern = re.compile("(.*(?:of|is)\s)(\w\w\w)$")
    # remove course_name
    pattern2 = re.compile("(.*)(\w\w\w\s?\d\d\d)(.*)")
    if pattern.match(entry[0]):
      # entry(entry[0], entry[1])
      entry = (re.sub(pattern, "\\1III", entry[0]), entry[1])
      cleaned.append(entry)
    elif pattern2.match(entry[0]):
      entry = (re.sub(pattern2, "\\1III000\\3", entry[0]), entry[1])
      cleaned.append(entry)
    else:
      cleaned.append(entry)
    # remove name (SVM required) #TODO
  return cleaned


cleanParams()

[('Email id of III', 'mail_id'),
 ('Email id of III', 'mail_id'),
 ('Email id of nazmus sakeef', 'mail_id'),
 ('Email id of III', 'mail_id'),
 ('Email id of III', 'mail_id'),
 ('Email id of szi?', 'mail_id'),
 ('email id of sro?', 'mail_id'),
 ('Mail id of salman sir?', 'mail_id'),
 ('Mail id of annajiat', 'mail_id'),
 ('Mail id of ishita miss?', 'mail_id'),
 ('Email id of mht?', 'mail_id'),
 ('email id of sro?', 'mail_id'),
 ('email id of szn?', 'mail_id'),
 ('mail id of III', 'mail_id'),
 ('email id of szn?', 'mail_id'),
 ('email id of III', 'mail_id'),
 ('Email id of zahid hussain mohammed al deen sir?', 'mail_id'),
 ('email id of mrn?', 'mail_id'),
 ('Email ID of III', 'mail_id'),
 ('Email ID of III', 'mail_id'),
 ('May I have the email ID of AHR?', 'mail_id'),
 ('Email ID of ARK!', 'mail_id'),
 ('May I have the email address of III', 'mail_id'),
 ('May I have the email address of III', 'mail_id'),
 ('Email id of III', 'mail_id'),
 ('Email id of III', 'mail_id'),
 ('Email id of III

In [ ]:
just_questions = list(map(lambda x: x[0], cleanParams()))

# creating vocabulary
joint = " ".join(just_questions)
wordList = joint.split(" ")
# remove question character
wordSet = set(wordList)
words = list(wordSet) # vocab
print(len(joint))
print(len(wordList))
print(len(wordSet))

6445
1413
177


* Vocabulary ["mail", "id"....., "of", 99]
* "mail id of III" => [0, 1, 5, 99]

In [ ]:
# sample vectorization
def vectorize(sentence):
  vec = np.zeros(len(words))
  tokens = sentence.split(" ")
  for token in tokens:
    if token not in words:
      continue
    vec[words.index(token)] = 1
  return vec

vectorize("full name of shd")

array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 1., 0., 0., 0., 0., 0.])

# Classification

In [ ]:
# example SVM = Support Vector Machine
from sklearn import svm
X = [[0, 0], [1, 1], [2, 3], [5, 5]]
y = [0, 1, 1, 0]
clf = svm.SVC(kernel = 'linear')
clf.fit(X, y)

print(clf.predict([[1, 5]]))

[1]


In [ ]:
from sklearn import svm
import numpy as np
from sklearn.model_selection import train_test_split

just_response = list(map(lambda x: x[1], cleanParams()))



X = list(map(lambda sent: vectorize(sent), just_questions))
y = just_response
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7,test_size=0.3, random_state=1)
clf = svm.SVC(kernel='linear').fit(X_train, y_train)
clf.score(X_test, y_test)

0.9629629629629629

# Test bot

In [ ]:
# TODO lemmatize course/courses
# was/is/are => be
sentence = "my name is robin"
p_type = clf.predict([vectorize(sentence)])
p_type

array(['full_name'], dtype='<U13')

In [ ]:
# extract initial or other param
